In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms


In [132]:
input_size = 3*32*32   # Размер изображения в точках * количество цветов
num_classes = 10       # Количество распознающихся классов (10 видов изображений)
n_epochs = 10          # Количество эпох
batch_size = 4         # Размер мини-пакета входных данных
lr = 0.001             # Скорость обучения

In [3]:
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar_trainset = dsets.CIFAR10(root='./data', train=True, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:02<00:00, 78741010.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [4]:
cifar_testset = dsets.CIFAR10(root='./data', train=False, download=True, transform=transform)
print(len(cifar_trainset))
print(len(cifar_testset))

Files already downloaded and verified
50000
10000


In [5]:
train_loader = torch.utils.data.DataLoader(dataset=cifar_trainset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=cifar_testset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [6]:
# импортируем нужные библиотеки
import torch
import numpy as np # всегда пригодится :)
from torch.nn import Linear, Sigmoid

# инициализируем девайс
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# добавляем типовую функцию "шаг обучения"
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

In [138]:
import torch.nn as nn
import torch.nn.functional as F


class CifarModel(nn.Module):
    def __init__(self):
        super(CifarModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.drop = nn.Dropout(0.1)
        self.fc1 = nn.Linear(16 * 3 * 3 * 4, 2* 120)
        self.fc2 = nn.Linear(2 * 120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.drop(x)
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 16 * 3 * 3 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [139]:
from torch import optim, nn

model = CifarModel()
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        loss = train_step(images, labels)
    print(epoch)

# print(model.state_dict())
print(loss)

0
1
2
3
4
5
6
7
8
9
1.2384783029556274


In [134]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4):
    print('Точность для %5s : %2d %%' % (
        labels[i], 100 * class_correct[i] / class_total[i]))

Точность для tensor(3) : 65 %
Точность для tensor(5) : 74 %
Точность для tensor(1) : 44 %
Точность для tensor(7) : 42 %


In [140]:
with torch.no_grad(): # проверяем на тестовой выборке
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Точность: {} %'.format(100 * correct / total))

Точность: 63.6 %
